Extract quantities of interest for initializing N-body intitial conditons

Requirements:
 - an environment capable of running SatGen (although we don't run SatGen itself, the SatGen routines we use depend on various other packages).
 - an "updated" version of SatGen compatible with recent versions of `numpy`.
 - `pytables` (could be replaced with h5py by editing the read_hdf5 function below)

In [1]:
pwd

'/data/apcooper/projects/sgarrak/notebooks/ics_example'

In [2]:
import sys
import os
import time

import numpy as np
import tables as tb
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import astropy.cosmology as cosmo

from astropy.table import Table

import copy

SATGEN_PATH = '/data/apcooper/sfw/SatGen'
if not SATGEN_PATH in sys.path:
    sys.path.append(SATGEN_PATH)

SATGEN_ETC_PATH = '/data/apcooper/sfw/SatGen/etc'
if not SATGEN_ETC_PATH in sys.path:
    sys.path.append(SATGEN_ETC_PATH)
        
# SatGen Imports
import config as cfg
import cosmo as co
import evolve as ev
from   profiles import NFW,Dekel,MN,Einasto,Green
from   orbit import orbit
import galhalo as gh
import aux
import init

>>> Normalizing primordial power spectrum P(k)=(k/k_0)^n_s ...
    such that sigma(R=8Mpc/h) =   0.8000.
>>> Building interpolation grid for Green+19 M(<r|f_b,c)...
>>> Building interpolation grid for Green+19 sigma(r|f_b,c)...
>>> Building interpolation grid for Green+19 d2Phidr2(r|f_b,c)...
>>> Building interpolator for Jiang+15 orbit sampler...
[0.  0.5 1.  1.5]


In [3]:
import matplotlib.pyplot as pl
%matplotlib inline

In [4]:
def read_hdf5(path,datasets,group='/'):
    with tb.open_file(path, 'r') as f:
        if isinstance(datasets,str):
            # Just one dataset, read it as an array
            data = f.get_node(f'{group}/{datasets}').read()
        else:
            # Assume nodes is an iterable of dataset names under group
            data = dict([ (name,f.get_node(f'{group}/{name}').read()) for name in datasets])
    return data

### Read PCHTrees output

In [5]:
# Millennium Cosmology (should match whatever was used in PCHTrees)
hubble_parameter = 0.73
cosmology = cosmo.FlatLambdaCDM(hubble_parameter*100,0.25)

This notebook will use the example tree run in `./trees/`, under the directory containing this notebook. After compiling `pchtrees`, follow the README.md in that directory to create the trees.

This should generate `./trees/output_progenitors_100_mw.hdf5`. This file contains only the "first order progenitors" and main branch mass histories, not the full trees.

In [6]:
# Output of PCHTrees
tree_file = './trees/output_progenitors_100_mw.hdf5'

# Read main branch mass histories (immediately deal with little h)
tree_main_branch_masses = read_hdf5(tree_file,'/Mainbranch/MainbranchMass')/hubble_parameter

# Number of treees and tree levels
ntrees, nlev = tree_main_branch_masses.shape

# Read progenitor data (immediately deal with little h on masses)
progenitor_dataset_names = ["HostMass","ProgenitorZred","ProgenitorMass","ProgenitorIlev","TreeID"]
    
progenitors = read_hdf5(tree_file, progenitor_dataset_names, group='/Progenitors')
progenitors['ProgenitorMass'] = progenitors['ProgenitorMass']/hubble_parameter
progenitors['HostMass']       = progenitors['HostMass']/hubble_parameter

tree_redshifts = read_hdf5(tree_file,'Redshift',group='/OutputTimes')
tree_t_lbk_gyr = cosmology.lookback_time(tree_redshifts).value
tree_t_age_gyr = cosmology.age(tree_redshifts).value

# Root mass is the the mass of the main branch at z=0
root_mass = tree_main_branch_masses[:,0]

### Extract N-body ICs

Since we're not going to run the SatGen evolution, we just need to extract the equivalent quantities from the tree files. We have to do a small amount of processing that paralells the setup in SatGen for things like orbits, stellar masses and concentrations.

Really this should all be integrated in one code to avoid inconsistency, but the point here is to show the nuts and bolts.

In [7]:
tree_main_branch_masses.shape

(1, 128)

In [8]:
print(f'There are {tree_main_branch_masses.shape[0]:d} merger trees in the file.')
print(f'There are {tree_main_branch_masses.shape[1]:d} time levels in each tree.')

There are 1 merger trees in the file.
There are 128 time levels in each tree.


We first select a tree, and find all the progenitors in that tree.

In [9]:
itree = 0

We have to be careful to set the concentration of the host consistently along the main branch, so that all progenitors use the same host concentration (as opposed to randomly drawing the host concentration for each merger event)

In [10]:
def halo_mah_to_zhao_c_nfw(mass, t_age_gyr):
    """
    """
    h_c_nfw = list()
    nlev = np.atleast_1d(mass).shape[0]
    for i in range(0,nlev):
        h_c_nfw.append(init.c2_fromMAH(np.atleast_1d(mass)[i:],np.atleast_1d(t_age_gyr)[i:]))
    return np.array(h_c_nfw)

In [11]:
# Note that we pass arrays here, and get an array back: one concentration for the host at each level.
host_c = halo_mah_to_zhao_c_nfw(tree_main_branch_masses[itree,:], tree_t_age_gyr)

We now proceed to choose ICs for a specific progenitor. We first identify all the progenitors in the selected tree:

In [12]:
prog_this_tree = np.flatnonzero(progenitors['TreeID'] == itree)
print(f'There are {len(prog_this_tree):d} progenitors in tree {itree:d}.')

There are 376 progenitors in tree 0.


To illustrate the process, we select only one progenitor. For experiments with single progenitors, it helps to know which progenitors are the most massive.

In [13]:
prog_mass_order = np.argsort(progenitors['ProgenitorMass'])[::-1] # Largest first

The 10 most massive are:

In [14]:
prog_mass_order[0:10]

array([163, 343,  36, 241,  66, 195, 309, 335, 189, 205])

In [15]:
iprog = 163

Now extract properties for this progenitor:

In [16]:
# Infall redshift 
prog_zred = progenitors['ProgenitorZred'][prog_this_tree][iprog]
# Infall level
prog_ilev = progenitors['ProgenitorIlev'][prog_this_tree][iprog]
# Mass (Msol)
prog_mass = progenitors['ProgenitorMass'][prog_this_tree][iprog]

To draw orbital properties for the progenitor, we also need the mass of the host at the time of infall:

In [17]:
host_mass = tree_main_branch_masses[itree,prog_ilev]

In [18]:
print(f'At the time of infall (z = {prog_zred:<5.3f}), the halo masses are:')
print(f'Host:       M200c = {host_mass:7.2e} Msol')
print(f'Progenitor: M200c = {prog_mass:7.2e} Msol')

At the time of infall (z = 1.449), the halo masses are:
Host:       M200c = 1.06e+12 Msol
Progenitor: M200c = 7.64e+10 Msol


We can initialize the progenitor concentration, using the DM14 distribution in SatGen:

In [19]:
prog_c = init.concentration(prog_mass,prog_zred,choice='DM14')
print(f'Prog: c = {prog_c:5.3f}')

Prog: c = 6.461


We can assign a stellar mass to the progenitor using the Behroozi 13 relation (from SatGen):

In [20]:
prog_mstar = init.Mstar(prog_mass, prog_zred, choice='B13')
print(f'Prog: Mstar = {prog_mass:5.3g} Msol')

Prog: Mstar = 7.64e+10 Msol


To initialize the orbit using SatGen, we first have to create a SatGen "DensityProfile" object for the host:

In [21]:
host_profile = NFW(host_mass, host_c[prog_ilev], Delta=200, z=prog_zred)

Note that we use prog_ilev to extract the concentration of the host.

Now use SatGen to set up the orbit.

In [22]:
vel_ratio, gamma = init.ZZLi2020(host_profile, prog_mass, prog_zred)
xv = init.orbit_from_Li2020(host_profile, vel_ratio, gamma)

`xv` contains the phase space coordinates of the progenitor in cylindrical coordinates, i.e. $(R,\theta,Z, v_R, v_\theta, v_Z)$. The `R` coordinate is in the disk plane (in this case hypothetical, since there is no disk). I think the halo is always spherical in SatGen.

The angle $\theta$ is in radians, from 0 to $2\pi$.

I think $R^2 + Z^2 \equiv R_\mathrm{vir}^2$ by construction, i.e. these orbital properties are defined at $R_\mathrm{vir}$. But see SatGen papers for details, I suppose.

In summary, the properties of interest are:

In [25]:
host_mass, host_c[prog_ilev], prog_mass, prog_c, prog_mstar, prog_zred, xv

(1056351100000.0,
 4.046165222743265,
 76351410000.0,
 6.461338690623547,
 71722798.38227697,
 1.4488189,
 array([  57.86201032,    3.05089878,  107.41808053,  -75.57956348,
         -31.87047875, -154.42320111]))